In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)

c:\Users\PC\Data\AI_project_final


In [2]:
import functools
from vnstock3 import Vnstock
import json
import numpy as np
import pandas as pd
from langchain_core.runnables import RunnablePassthrough
# from models.lstm_stock_price import model
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage
from langchain_community.agent_toolkits import create_sql_agent
# from autogen import RAGAgent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
# from langchain.retrievers import ChromaRetriever
# from langchain.embeddings import ChromaEmbedder
from langchain import LLMChain
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
import getpass
import os
from dotenv import load_dotenv, find_dotenv
from langchain.sql_database import SQLDatabase
from langgraph.prebuilt import ToolNode
from typing import Literal, List
from datetime import datetime, timedelta
from scipy.optimize import minimize
load_dotenv(find_dotenv())

True

In [3]:
# os.environ["OPENAI_API_KEY"] = getpass.getpass("Please provide your OPENAI_API_KEY:")
# os.environ["POSTGRESQL_DBMS_KEY"] = getpass.getpass("Please provide your POSTGRESQL_DBMS_KEY:")
llm = ChatOpenAI(model="gpt-4-1106-preview" , api_key=os.getenv("OPEN_API_KEY"))

db = SQLDatabase.from_uri(os.getenv('POSTGRESQL_DBMS_KEY'))
db

In [33]:
###EUT
@tool
def calculate_utility(outcome: float, risk_aversion: float) -> float:
    """
    Tính toán độ hữu ích dựa trên kết quả và hệ số ngại rủi ro.

    Args:
        outcome: Kết quả tài chính cần đánh giá.
        risk_aversion: Hệ số ngại rủi ro.
    """
    return outcome ** (1 - risk_aversion) / (1 - risk_aversion)
@tool
def calculate_expected_utility(probabilities: List[float], utilities: List[float]) -> float:
    """
    Tính toán độ hữu ích kỳ vọng.

    Args:
        probabilities:Danh sách xác suất liên quan đến các kết quả khác nhau.
        utilities: Danh sách độ hữu ích tương ứng với các kết quả.

    Raises:
        ValueError:Nếu độ dài của probabilities và utilities không khớp nhau.
    """
    if len(probabilities) != len(utilities):
        raise ValueError("Probabilities and utilities must have the same length.")
    
    expected_utility = np.dot(probabilities, utilities)
    return expected_utility


# @tool
# def porfolio_optimize_EUT():
#     pass

###MVP
@tool
def calculate_portfolio_return(returns: List[float], weights: List[float]) -> float:
    """
    Tính toán lợi nhuận hàng năm của danh mục đầu tư.

    Args:
        returns: Danh sách lợi nhuận lịch sử của các tài sản trong danh mục đầu tư.
        weights: Danh sách tỷ trọng đại diện cho phần trăm của từng tài sản trong danh mục.
    """
    return np.dot(returns.mean(), weights) * 252

@tool
def calculate_portfolio_volatility(returns: List[float], weights: List[float]) -> float:
    """
    Tính toán biến động hàng năm của danh mục đầu tư.

    Args:
        returns:  Danh sách lợi nhuận lịch sử của các tài sản trong danh mục đầu tư.
        weights:  Danh sách tỷ trọng đại diện cho phần trăm của từng tài sản trong danh mục.
    """
    return np.dot(weights, np.dot(returns.cov() * 252, weights)) ** 0.5  # Annualized volatility

@tool
def calculate_sharpe_ratio(returns: List[float], weights: List[float], risk_free_rate: float) -> float:
    """
    Tính toán tỷ lệ Sharpe của danh mục đầu tư.

    Args:
        returns: Danh sách lợi nhuận lịch sử của các tài sản trong danh mục đầu tư.
        weights: Danh sách tỷ trọng đại diện cho phần trăm của từng tài sản trong danh mục.
        risk_free_rate: Tỷ lệ lợi nhuận không rủi ro.
    """
    port_return = calculate_portfolio_return(returns, weights)
    port_volatility = calculate_portfolio_volatility(returns, weights)
    sharpe_ratio = (port_return - risk_free_rate) / port_volatility
    return sharpe_ratio

@tool
def portfolio_optimize(returns, sharpe_ratio_or_variance=True):
    """
    Hàm này dùng để tối ưu hóa danh mục đầu tư dựa trên tỷ lệ Sharpe hoặc biến động sau khi biết được tình hình doanh mục đầu tư được dự đoán trong tương lai trước đó.

    Args:
        returns: Thông tin tình hình danh mục đầu tư sau khi dùng hàm predict_future_prices .
        sharpe_ratio_or_variance: Một giá trị boolean chỉ định có tối ưu hóa cho tỷ lệ Sharpe (True) hay biến động (False).
    """
    returns = json.loads(returns)
    returns = pd.DataFrame([returns])
    # returns=returns[returns.columns[1:]]
    # print(returns)
    num_assets = len(returns.columns)
    # print('haha',num_assets)
    init_guess = num_assets * [1. / num_assets]
    bounds = tuple((0, 1) for asset in range(num_assets))
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    if sharpe_ratio_or_variance:
        result = minimize(lambda x: -calculate_sharpe_ratio(x, returns), init_guess, method='SLSQP', bounds=bounds, constraints=constraints)
    else:
        result = minimize(lambda x: calculate_portfolio_volatility(x, returns), init_guess, method='SLSQP', bounds=bounds, constraints=constraints)
    
    # Chuyển đổi tỷ trọng tối ưu thành từ điển với tên cổ phiếu
    optimal_weights = dict(zip(returns.columns, result.x))
    
    return optimal_weights

### CAPM
@tool
def calculate_beta(asset_returns: List[float], market_returns: List[float]) -> float:
    """
    Tính toán hệ số beta của một tài sản.

    Args:
        asset_returns: Danh sách lợi nhuận của tài sản.
        market_returns: Danh sách lợi nhuận của thị trường.
    """
    covariance_matrix = np.cov(asset_returns, market_returns)
    beta = covariance_matrix[0, 1] / covariance_matrix[1, 1]
    return beta

@tool
def calculate_capm(risk_free_rate: float, beta: float, market_return: float) -> float:
    """
    Tính toán lợi nhuận kỳ vọng của một tài sản bằng công thức CAPM.

    Args:
        risk_free_rate: Tỷ lệ lợi nhuận không rủi ro.
        beta: Hệ số beta của tài sản.
        market_return: Lợi nhuận kỳ vọng của thị trường.
    """
    expected_return = risk_free_rate + beta * (market_return - risk_free_rate)
    return expected_return


#### Make prediction
@tool
def predict_future_prices(symbols, n):
    """
    Dự đoán giá cổ phiếu trong tương lai dựa trên dữ liệu đầu vào. Phù hợp cho việc dự báo tình hình của danh mục 

    Args:
        symbols: Danh sách mã cổ phiếu cần dự đoán.
        n: Số tháng từ ngày hiện tại để dự đoán.
    """
    data = get_api_portfolio(symbols)

    try:
        df = pd.DataFrame(eval(data))
    except (ValueError, SyntaxError) as e:
        print(f"Error creating DataFrame: {e}")
        return
    
    if df.empty or not {'idx_time', 'AAA', 'A32'}.issubset(df.columns):
        print("Data không hợp lệ. Đảm bảo dữ liệu chứa các cột 'idx_time', 'AAA', và 'A32'.")
        return
    
    df['idx_time'] = pd.to_datetime(df['idx_time'] + '-01', format='%Y-%m-%d')
    
    df['days'] = (df['idx_time'] - df['idx_time'].min()).dt.days
    
    df = df.dropna(subset=symbols)
    
    degree = 2
    coeffs = {}

    for symbol in symbols:
        coeffs[symbol] = np.polyfit(df['days'], df[symbol], degree)
    
    future_dates = pd.date_range(start=pd.to_datetime('today').replace(day=1), periods=n, freq='MS')
    future_days = (future_dates - df['idx_time'].min()).days
    
    predictions = {}
    for symbol in symbols:
        predictions[symbol] = np.polyval(coeffs[symbol], future_days)
    
    future_df = pd.DataFrame({
        'idx_time': future_dates.strftime('%Y-%m'),  
        **predictions
    })
    
    last_prediction = future_df.iloc[-1]
    
    result = {
        # 'idx_time': last_prediction['idx_time'],
        **{symbol: last_prediction[symbol] for symbol in symbols}
    }
    
    return json.dumps(result, default=str)

####Get API
@tool
def generate_prompt_with_date(prompt):
    """
    Tạo một prompt có kèm theo ngày hiện tại.

    Args:
        prompt: Chuỗi prompt ban đầu.
    """
    from datetime import datetime
    current_date = datetime.now().strftime("%Y-%m-%d")
    full_prompt = f"{prompt} Today's date is {current_date}."
    return full_prompt

@tool
def get_company_information(symbol):
    """
    Lấy thông tin công ty bao gồm tổng quan, hồ sơ, cổ đông, công ty con, và cán bộ. Phù hợp cho việc điều tra thông tin cụ thể của một doanh nghiệp trước khi đưa ra quyết định

    Args:
        symbol: Mã cổ phiếu của công ty.
    """
    company = Vnstock().stock(symbol=symbol, source='TCBS').company
    
    try:
        overview = company.overview()
        profile = company.profile()  # Assuming profile is another method; adjust if incorrect
        shareholders = company.shareholders()
        subsidiaries = company.subsidiaries()
        officers = company.officers()
        
        result = {
            "overview": overview,
            "profile": profile,
            "shareholders": shareholders,
            "subsidiaries": subsidiaries,
            "officers": officers
        }
    except Exception as e:
        result = {"error": str(e)}
    
    return json.dumps(result)


@tool
def get_api_stock(symbols, startday, endday):
    """
    Lấy dữ liệu cổ phiếu cho một mã và khoảng thời gian cụ thể.

    Args:
        symbols: Mã cổ phiếu cần lấy dữ liệu.
        startday: Ngày bắt đầu của khoảng thời gian.
        endday: Ngày kết thúc của khoảng thời gian.
    """


    stock = Vnstock().stock(symbol=symbols, source='TCBS')
    
    try:
        startday = pd.to_datetime(startday).strftime('%Y-%m-%d')
        endday = pd.to_datetime(endday).strftime('%Y-%m-%d')
        df = stock.quote.history(start=startday, end=endday, interval='1D')
        
        if not df.empty:
            df = df.map(lambda x: x.isoformat() if isinstance(x, (pd.Timestamp, pd.Timestamp)) else x)
            result = df.to_dict(orient='records')
        else:
            result = {"error": "Có lẽ ngày này ngân hàng không làm việc, do đây có thể là thứ bảy chủ nhật hoặc ngày lễ"}
        if startday==endday:
            result = [record for record in result if record['time'].startswith(startday)]
            if result.empty:
                result={"error": "Có lẽ ngày này ngân hàng không làm việc, do đây có thể là thứ bảy chủ nhật hoặc ngày lễ"}

    
    except Exception as e:
        result = {"error": "Có lẽ ngày này ngân hàng không làm việc, do đây có thể là thứ bảy chủ nhật hoặc ngày lễ"}

    
    return json.dumps(result, ensure_ascii=False)


@tool
def get_api_income_statement(symbols):
    """
    Truy xuất bảng báo cáo tài chính theo năm của từng doanh nghiệp cụ thể

    Args:
        symbols: Mã doanh nghiệp được niêm yết trên sàn chuwnsg khoán.
    """
    stock = Vnstock().stock(symbol=symbols, source='TCBS')
    try:
        df = stock.finance.income_statement(period='year', lang='en')
        if not df.empty:
            result = df.to_dict(orient='records')
        else:
            result = {"error": "No data found for the given range"}
    except Exception as e:
        result = {"error": str(e)}
    
    return json.dumps(result)

@tool 
def get_api_portfolio(symbols):
    """
    Lấy dữ liệu về một danh mục đầu tư gồm nhiều mã cổ phiếu khác nhau trong một khoảng thời gian

    Args:
        symbols: Một danh sách các mã cổ phiếu trong danh mục đầu tư
    """
    endday = datetime.now()
    startday = endday - timedelta(days=365)
    
    data = pd.DataFrame()
    for symbol in symbols:
        stock = Vnstock().stock(symbol=symbol, source='TCBS')
        try:
            df = stock.quote.history(start=startday.strftime('%Y-%m-%d'), 
                                     end=endday.strftime('%Y-%m-%d'), 
                                     interval='1D')
            if df.empty:
                return {"error": "No data found for symbol {}".format(symbol)}
            
            df['time'] = pd.to_datetime(df['time'])
            df = df.sort_values(by='time', ascending=False)
            
            df['month'] = df['time'].dt.month
            df['year'] = df['time'].dt.year
            
            df['idx_time'] = df['time'].dt.to_period('M')
                        
            df.set_index('idx_time', inplace=True)          
            df = df[['close']]
            df.columns = [symbol]
            data = pd.concat([data, df], axis=1)
        except Exception as e:
            return {"error": "Error for symbol {}: {}".format(symbol, str(e))}
    if data.empty:
        return {"error": "No data found for the given symbols and range"}
    
    data = data.groupby(data.index).mean()
    print(data)
    rets = np.log(data / data.shift(1)).dropna()
    
    result = rets.reset_index().to_dict(orient='records')
    
    return result

@tool
def get_api_balance_sheet(symbols):
    """
    Truy xuất bảng cân đối kế toán của từng doanh nghiệp cụ thể

    Args:
        symbols: Mã doanh nghiệp được định giá trên sàn chứng khoán
    """
    stock = Vnstock().stock(symbol=symbols, source='TCBS')
    try:
        df = stock.finance.balance_sheet(period='year', lang='en')
        if not df.empty:
            result = df.to_dict(orient='records')
        else:
            result = {"error": "No data found for the given range"}
    except Exception as e:
        result = {"error": str(e)}
    
    return json.dumps(result)


tools = [
    calculate_utility,
    calculate_expected_utility,
    calculate_portfolio_return,
    calculate_portfolio_volatility,
    calculate_sharpe_ratio,
    calculate_beta,
    calculate_capm,
    generate_prompt_with_date,
    get_company_information,
    get_api_stock,
    get_api_portfolio,
    get_api_income_statement,
    get_api_balance_sheet,
    predict_future_prices,
    portfolio_optimize,
]

llm_with_tools = llm.bind_tools(tools)


### Function calling

In [13]:
query = f"""Bạn là một chuyên gia chơi chứng khoán trong lĩnh vực kinh tế đầu tư tại thị trường chứng khoán Việt Nam. Hôm nay là ngày {datetime.now().strftime('%Y-%m-%d')} Sử dụng các tool được cung cấp như một ví dụ để đưa ra những lời khuyên hữu ích để người chơi mới tại Việt Nam lựa chọn và tối ưu hóa danh mục đầu tư, đầu tư chứng khoán thành công. Hiện nay, một danh mục đầu tư của tôi bao gồm các mã cổ phiếu của sàn VNINDEX như là AAM, XPH, YEG, AAT. Tôi nên phân bổ vốn của mình như nào để tối ưu hóa danh mục đầu tư sao cho lợi nhuận được tối đa nhất trong chiến lược đầu tư dài hạn 6 tháng tới như thế nào?
"""


messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

tool_mapping = {
    "calculate_utility": calculate_utility,
    "calculate_expected_utility": calculate_expected_utility,
    "calculate_portfolio_return": calculate_portfolio_return,
    "calculate_portfolio_volatility": calculate_portfolio_volatility,
    "calculate_sharpe_ratio": calculate_sharpe_ratio,
    "calculate_beta": calculate_beta,
    "calculate_capm": calculate_capm,
    "generate_prompt_with_date": generate_prompt_with_date,
    "get_company_information": get_company_information,
    "get_api_stock": get_api_stock,
    "get_api_income_statement": get_api_income_statement,
    "get_api_balance_sheet": get_api_balance_sheet,
    "predict_future_prices": predict_future_prices,
    "get_api_portfolio": get_api_portfolio,
    "portfolio_optimize": portfolio_optimize,
}


for tool_call in ai_msg.tool_calls:
    print(tool_call)
    selected_tool = tool_mapping[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

[HumanMessage(content='Bạn là một chuyên gia chơi chứng khoán trong lĩnh vực kinh tế đầu tư tại thị trường chứng khoán Việt Nam. Hôm nay là ngày 2024-08-27 Sử dụng các tool được cung cấp như một ví dụ để đưa ra những lời khuyên hữu ích để người chơi mới tại Việt Nam lựa chọn và tối ưu hóa danh mục đầu tư, đầu tư chứng khoán thành công. Hiện nay, một danh mục đầu tư của tôi bao gồm các mã cổ phiếu của sàn VNINDEX như là AAM, XPH, YEG, AAT. Tôi nên phân bổ vốn của mình như nào để tối ưu hóa danh mục đầu tư sao cho lợi nhuận được tối đa nhất trong chiến lược đầu tư dài hạn 6 tháng tới như thế nào?\n'),
 AIMessage(content='Để tối ưu hóa danh mục đầu tư của bạn trên sàn VNINDEX, chúng ta cần xem xét những yếu tố sau:\n\n1. Lịch sử biến động giá của các cổ phiếu trong danh mục: Nếu có dữ liệu về lịch sử giá cả và biến động của các mã cổ phiếu AAM, XPH, YEG, AAT, chúng ta có thể phân tích và dự đoán xu hướng trong tương lai.\n\n2. Đánh giá rủi ro và lợi nhuận kỳ vọng: Sử dụng phương pháp định

In [14]:
selected_tool

<function __main__.get_api_stock(symbols, startday, endday)>

In [15]:
llm_with_tools.invoke(messages)


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XnSP6FhiD46NeJJOdyEoELSl', 'function': {'arguments': '{"symbols":["AAM","XPH","YEG","AAT"],"startday":"2023-02-27","endday":"2023-08-27"}', 'name': 'get_api_stock'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 2064, 'total_tokens': 2109}, 'model_name': 'gpt-4-1106-preview', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cf9e428d-e6ac-4c60-a578-7a3d2d13bf50-0', tool_calls=[{'name': 'get_api_stock', 'args': {'symbols': ['AAM', 'XPH', 'YEG', 'AAT'], 'startday': '2023-02-27', 'endday': '2023-08-27'}, 'id': 'call_XnSP6FhiD46NeJJOdyEoELSl', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2064, 'output_tokens': 45, 'total_tokens': 2109})

### Few-short prompting

In [34]:
examples = [
    # HumanMessage(content="Mã cổ phiếu của công ty AAA ngày hôm nay đang ở mức bao nhiêu", name="example_user"),
    # AIMessage(content="", name="example_assistant", tool_calls=[{"name": "symbols", "args": {"symbols": "AAA", "startday": datetime.now().strftime('%Y-%m-%d'), "endday":datetime.now().strftime('%Y-%m-%d')}, "id": "1"}]),
    # ToolMessage(content='Có lẽ ngày này ngân hàng không làm việc, do đây có thể là thứ bảy chủ nhật hoặc ngày lễ', tool_call_id="1"),
    # AIMessage(content="Thị trường chứng khoán ngày giao dịch hôm nay hiện tại không hoạt động, nên chúng tôi không thể kiểm tra hoặc phân tích dữ liệu cho bạn được, nếu có thể bạn vui lòng quay lại vào ngày mai", name="example_assistant"),
    
    HumanMessage(content="Hiện nay, một danh mục đầu tư của tôi bao gồm các mã cổ phiếu của sàn VNINDEX như là AAM, XPH, YEG, AAT. Tôi nên phân bổ vốn của mình như nào để tối ưu hóa danh mục đầu tư sao cho lợi nhuận được tối đa nhất trong chiến lược đầu tư dài hạn 6 tháng tới như thế nào?", name="example_user1"),
    AIMessage(content="", name="example_assistant1", tool_calls=[{"name": "predict_future_prices", "args": {"symbols": ['AAM', 'XPH', 'YEG', 'AAT'], "n": 6}, "id": "1"}]),
    ToolMessage(content="{'AAM': 123, 'XPH': 123, 'YEG': 12, 'AAT': 134}", tool_call_id="1"),
    AIMessage(content="", name="example_assistant1", tool_calls=[{"name": "portfolio_optimize", "args": {"return": {'AAM': 123, 'XPH': 123, 'YEG': 12, 'AAT': 134}, "sharpe_ratio_or_variance": True}, "id": "2"}]),
    ToolMessage(content="{'AAM': 0.25, 'XPH': 0.25, 'YEG': 0.25, 'AAT': 0.25}", tool_call_id="2"),
    AIMessage(content="Có thể thấy, sau khi chạy mô hình dự đoán của hệ thống, tình hình cổ phiếu sẽ có sự biến động khá cao. Như vậy dựa vào kết quả dự đoán tình hình từng mã sau 6 tháng tới, bạn cần phân bố đều vốn cá nhân 25% ứng với mỗi loại cổ phiếu, để có được lợi nhuận cao nhất", name="example_assistant1"),

    # HumanMessage(content="Hiện nay, một danh mục đầu tư của tôi bao gồm các mã cổ phiếu của sàn VNINDEX như là AAM, XPH, YEG, AAT. Tôi nên phân bổ vốn của mình như nào để tối ưu hóa danh mục đầu tư sao giảm thiểu rủi ro nhất trong chiến lược đầu tư dài hạn 6 tháng tới như thế nào?", name="example_user2"),
    # AIMessage(content="", name="example_assistant2", tool_calls=[{"name": "predict_future_prices", "args": {"symbols": ['AAM', 'XPH', 'YEG', 'AAT'], "n": 6}, "id": "1"}]),
    # ToolMessage(content=json.dumps({'AAM': 123, 'XPH': 123, 'YEG': 12, 'AAT': 134}), tool_call_id="1"),
    # AIMessage(content="", name="example_assistant2", tool_calls=[{"name": "portfolio_optimize", "args": {"return": {'AAM': 123, 'XPH': 123, 'YEG': 12, 'AAT': 134}, "sharpe_ratio_or_variance": False}, "id": "2"}]),
    # ToolMessage(content=json.dumps({'AAM': 0.15, 'XPH': -0.05, 'YEG': 0.45, 'AAT': 0.45}), tool_call_id="2"),
    # AIMessage(content="Có thể thấy, sau khi chạy mô hình dự đoán của hệ thống, tình hình cổ phiếu sẽ có sự biến động khá cao. Như vậy dựa vào kết quả dự đoán tình hình từng mã sau 6 tháng tới, bạn cần phân bố đều vốn cá nhân lần lượt là, đầu tiên, rút 5% vốn hiện tại khỏi XPH, sử dụng 15% vốn vào AAM, 45% vốn cho cả YEG và AAT ứng với mỗi loại cổ phiếu, để có được lợi nhuận cao nhất", name="example_assistant2"),

    # HumanMessage(content="Đánh giá công ty mã AAA có hoạt động ổn định hay không trong suốt 6 tháng qua, check lại sự kiện kèm tin tức để đánh giá mức độ của công ty.", name="example_user3"),
    # AIMessage(content="Để đánh giá công ty, tôi sẽ kiểm tra các chỉ số tài chính chính của công ty trong 6 tháng qua và xem xét các sự kiện quan trọng, cũng như tin tức ảnh hưởng đến công ty. Hãy chờ trong giây lát..."),
    # AIMessage(content="", name="example_assistant3", tool_calls=[{"name": "get_company_information", "args": {"symbol": "AAA"}, "id": "1"}]),
    # ToolMessage(content="Thông tin công ty mã AAA và các sự kiện quan trọng đã được thu thập.", tool_call_id="1"),
    # AIMessage(content="Sau khi kiểm tra, tôi thấy rằng công ty mã AAA có tình hình hoạt động tương đối ổn định trong 6 tháng qua, tuy nhiên, có một số sự kiện và tin tức quan trọng cần lưu ý...", name="example_assistant3"),
]


In [35]:
# llm = ChatOpenAI(model="gpt-4-1106-preview" , api_key=os.getenv("OPEN_API_KEY"))
# llm_with_tools = llm.bind_tools(tools)

system=f"""Bạn là một chuyên gia chơi chứng khoán trong lĩnh vực kinh tế đầu tư tại thị trường chứng khoán Việt Nam. Hôm nay là ngày {datetime.now().strftime('%Y-%m-%d')}. Sử dụng các tool được cung cấp như một ví dụ để đưa ra những lời khuyên hữu ích để người chơi mới tại Việt Nam lựa chọn và tối ưu hóa danh mục đầu tư, đầu tư chứng khoán thành công
"""
few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        *examples,
        ("human", "{query}"),
    ]
    
)

chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools
chain.invoke("Hiện nay, một danh mục đầu tư của tôi bao gồm các mã cổ phiếu của sàn VNINDEX như là AAM, XPH, YEG, AAT. Tôi nên phân bổ vốn của mình như nào để tối ưu hóa danh mục đầu tư sao cho lợi nhuận được tối đa nhất trong chiến lược đầu tư dài hạn 6 tháng tới như thế nào?").tool_calls

[{'name': 'predict_future_prices',
  'args': {'symbols': ['AAM', 'XPH', 'YEG', 'AAT'], 'n': 6},
  'id': 'call_BDFB1qqyzGIYbhKJsYaDHzaG',
  'type': 'tool_call'}]

In [36]:
chain.invoke("Hiện nay, một danh mục đầu tư của tôi bao gồm các mã cổ phiếu của sàn VNINDEX như là AAM, XPH, YEG, AAT. Tôi nên phân bổ vốn của mình như nào để tối ưu hóa danh mục đầu tư sao cho lợi nhuận được tối đa nhất trong chiến lược đầu tư dài hạn 6 tháng tới như thế nào?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tjLby3qqexjEqSVWyM46NQ24', 'function': {'arguments': '{"symbols":["AAM","XPH","YEG","AAT"],"n":6}', 'name': 'predict_future_prices'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 2324, 'total_tokens': 2353}, 'model_name': 'gpt-4-1106-preview', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-eb0248a7-c6ad-4601-b3b1-96faedacfc5d-0', tool_calls=[{'name': 'predict_future_prices', 'args': {'symbols': ['AAM', 'XPH', 'YEG', 'AAT'], 'n': 6}, 'id': 'call_tjLby3qqexjEqSVWyM46NQ24', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2324, 'output_tokens': 29, 'total_tokens': 2353})

In [8]:
chain.invoke("tình hình hiện tại cổ phiếu AAA nay giá bao nhiêu")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nYmLCGWTw5TZKZIFWC8QXlaa', 'function': {'arguments': '{"symbols":"AAA","startday":"2024-08-26","endday":"2024-08-26"}', 'name': 'get_api_stock'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 2356, 'total_tokens': 2391}, 'model_name': 'gpt-4-1106-preview', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1c98e351-1eb7-482b-a329-f428f21b11fe-0', tool_calls=[{'name': 'get_api_stock', 'args': {'symbols': 'AAA', 'startday': '2024-08-26', 'endday': '2024-08-26'}, 'id': 'call_nYmLCGWTw5TZKZIFWC8QXlaa', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2356, 'output_tokens': 35, 'total_tokens': 2391})

In [8]:
# # Giả sử bạn đã gọi công cụ và nhận được phản hồi
# def process_tool_response(response):
#     if 'tool_calls' in response:
#         for tool_call in response['tool_calls']:
#             print(f"Công cụ: {tool_call['name']}")
#             print(f"Tham số: {tool_call['args']}")
#     else:
#         print("Phản hồi không chứa dữ liệu từ công cụ.")

# # Xử lý phản hồi
# process_tool_response(response)


In [9]:
def get_api_stock(symbols, startday, endday):

    stock = Vnstock().stock(symbol=symbols, source='TCBS')
    
    try:
        startday = pd.to_datetime(startday).strftime('%Y-%m-%d')
        endday = pd.to_datetime(endday).strftime('%Y-%m-%d')
        df = stock.quote.history(start=startday, end=endday, interval='1D')
        
        if not df.empty:
            df = df.map(lambda x: x.isoformat() if isinstance(x, (pd.Timestamp, pd.Timestamp)) else x)
            result = df.to_dict(orient='records')
        else:
            result = {"error": "Có lẽ ngày này ngân hàng không làm việc, do đây có thể là thứ bảy chủ nhật hoặc ngày lễ"}
        if startday==endday:
            result = [record for record in result if record['time'].startswith(startday)]
            if result.empty:
                result={"error": "Có lẽ ngày này ngân hàng không làm việc, do đây có thể là thứ bảy chủ nhật hoặc ngày lễ"}

    
    except Exception as e:
        result = {"error": "Có lẽ ngày này ngân hàng không làm việc, do đây có thể là thứ bảy chủ nhật hoặc ngày lễ"}

    
    return json.dumps(result, ensure_ascii=False)
stock_data = get_api_stock(symbols="AAA", startday=datetime.now().strftime('%Y-%m-%d'), endday=datetime.now().strftime('%Y-%m-%d'))
stock_data

'{"error": "Có lẽ ngày này ngân hàng không làm việc, do đây có thể là thứ bảy chủ nhật hoặc ngày lễ"}'